# Data Preprocessing

Data preprocessing is a crucial step in any data analysis or visualization project, and this notebook is specifically designed to tackle the preprocessing of job listings obtained from LinkedIn. The ultimate objective is to transform the raw data into a format suitable for utilization in Tableau, a powerful data visualization tool.

* [Merging job listing pages](#merge)
* [Splitting company details](#company)
* [Splitting job details](#job)
* [Cleaning the list of skills](#skills)
* [Removing duplicates](#duplicates)
* [Saving job listings file](#save)
* [Listing associate skills](#list)

In [1]:
import pandas as pd
import os

### Merging job listing pages <a class="anchor" id="merge"></a>

* To streamline the data wrangling process, we consolidate the job listings that were extracted and saved as separate files for each page into a unified data frame. Data frames streamline diverse data handling in preprocessing by offering a structured and unified framework.

In [2]:
def merge_pages(job):
    folder_path = os.path.join(os.getcwd(), f'Datasets/{job}')
    csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]
    
    datasets = []
    for csv_file in csv_files:
        file_path = os.path.join(folder_path, csv_file)
        current_df = pd.read_csv(file_path)
        datasets.append(current_df)
    
    data = pd.concat(datasets, ignore_index=True)
    data['Classification'] = job
        
    return data

In [3]:
dataAnalyst_listings = merge_pages("Data Analyst")
dataEngineer_listings = merge_pages("Data Engineer")
dataScientist_listings = merge_pages("Data Scientist")

combined_data = pd.concat([dataAnalyst_listings, dataEngineer_listings, dataScientist_listings], ignore_index=True)
combined_data

,Title,CompanyDetails,JobDetails,JobDescription,JobSkills,Classification
0,Data and Reports Analyst,"Dynata · Davao, Davao Region, Philippines Repo...",Full-time Mid-Senior level,About the job\nResponsibilities for Reporting ...,"Data Analysis,Critical Thinking\nAdd,Data Anal...",Data Analyst
1,Data Analyst,"Arcadis · Makati, National Capital Region, Phi...",Hybrid Full-time Mid-Senior level,About the job\nRole description:\nMainly provi...,"Data Analysis,Analytical Skills\nAdd,DAX\nAdd,...",Data Analyst
2,Data Analyst - People Services,"GHD · Cebu, Central Visayas, Philippines Repos...",Hybrid Full-time Mid-Senior level,"About the job\nJob Description\n\nAt GHD, we d...","Analytics\nAdd,Change Impact Analysis\nAdd,Dat...",Data Analyst
3,Insights Analyst,"Isentia · Pasig, National Capital Region, Phil...",Hybrid Full-time Associate,About the job\nIsentia is the Asia-Pacific reg...,"Data Analysis,Google Sheets\nAdd,Google Worksp...",Data Analyst
4,Risk Data Analyst l Day Shift,"MicroSourcing · National Capital Region, Phili...",Remote Full-time Mid-Senior level,"About the job\nWith over 8,000 professionals a...","Data Analysis,Fraud Analysis\nAdd,Risk Analysi...",Data Analyst
...,...,...,...,...,...,...
692,Senior IT Officer (Data Scientist),"Asian Development Bank (ADB) · Manila, Nationa...",On-site Full-time Mid-Senior level,About the job\nImportant Information\n\nThis i...,"ADB\nAdd,Communication\nAdd,Communication Trai...",Data Scientist
693,Data Scientist,"Standard Chartered Bank · Makati, National Cap...",On-site Full-time Entry level,About the job\nRole Responsibilities\n\nWe are...,"Artificial Intelligence (AI)\nAdd,Business Req...",Data Scientist
694,Data Scientist,Staffing Ninja · Philippines 1 week ago · 18 a...,Remote Full-time Mid-Senior level,About the job\nRoles and Responsibilities\n● D...,"Applications Delivery\nAdd,Cascading Style She...",Data Scientist
695,Machine Learning Specialist (Python),"OpenText · Makati, National Capital Region, Ph...",On-site Full-time Mid-Senior level,About the job\nThe opportunity:\n\nWe are look...,"Data Analysis,Analytics\nAdd,Artificial Intell...",Data Scientist


### Splitting company details <a class="anchor" id="company"></a>

* The _CompanyDetails_ column contains the company name, location, and when the job was posted. In this case, we only need the name and location.
* The location also does not follow a consistent format. To address this issue, we have implemented standardization measures to ensure consistency across all entries.

In [4]:
split_data = combined_data['CompanyDetails'].str.split(" · ", expand=True)
# Extract company name
combined_data['Company'] = split_data[0]
# Separate location from when job is posted
combined_data['Location'] = split_data[1].str.split('Philippines').str[0]
# Ensures consistent format when location is in any Metro city
combined_data['Location'] = combined_data['Location'].apply(lambda x: ' '.join(x.split()[:2]) + "," if x.startswith('Metro') else x)
combined_data['Location'] = combined_data['Location'] + " Philippines"
combined_data['Location'] = combined_data['Location'].apply(lambda x: x.split(', ')[0] if x.split(', ')[0] == x.split(', ')[-1] else ', '.join([x.split(', ')[0], x.split(', ')[-1]]))
# Drop the original CompanyDetails column
combined_data = combined_data.drop('CompanyDetails', axis=1)
combined_data

,Title,JobDetails,JobDescription,JobSkills,Classification,Company,Location
0,Data and Reports Analyst,Full-time Mid-Senior level,About the job\nResponsibilities for Reporting ...,"Data Analysis,Critical Thinking\nAdd,Data Anal...",Data Analyst,Dynata,"Davao, Philippines"
1,Data Analyst,Hybrid Full-time Mid-Senior level,About the job\nRole description:\nMainly provi...,"Data Analysis,Analytical Skills\nAdd,DAX\nAdd,...",Data Analyst,Arcadis,"Makati, Philippines"
2,Data Analyst - People Services,Hybrid Full-time Mid-Senior level,"About the job\nJob Description\n\nAt GHD, we d...","Analytics\nAdd,Change Impact Analysis\nAdd,Dat...",Data Analyst,GHD,"Cebu, Philippines"
3,Insights Analyst,Hybrid Full-time Associate,About the job\nIsentia is the Asia-Pacific reg...,"Data Analysis,Google Sheets\nAdd,Google Worksp...",Data Analyst,Isentia,"Pasig, Philippines"
4,Risk Data Analyst l Day Shift,Remote Full-time Mid-Senior level,"About the job\nWith over 8,000 professionals a...","Data Analysis,Fraud Analysis\nAdd,Risk Analysi...",Data Analyst,MicroSourcing,"National Capital Region, Philippines"
...,...,...,...,...,...,...,...
692,Senior IT Officer (Data Scientist),On-site Full-time Mid-Senior level,About the job\nImportant Information\n\nThis i...,"ADB\nAdd,Communication\nAdd,Communication Trai...",Data Scientist,Asian Development Bank (ADB),"Manila, Philippines"
693,Data Scientist,On-site Full-time Entry level,About the job\nRole Responsibilities\n\nWe are...,"Artificial Intelligence (AI)\nAdd,Business Req...",Data Scientist,Standard Chartered Bank,"Makati, Philippines"
694,Data Scientist,Remote Full-time Mid-Senior level,About the job\nRoles and Responsibilities\n● D...,"Applications Delivery\nAdd,Cascading Style She...",Data Scientist,Staffing Ninja,Philippines
695,Machine Learning Specialist (Python),On-site Full-time Mid-Senior level,About the job\nThe opportunity:\n\nWe are look...,"Data Analysis,Analytics\nAdd,Artificial Intell...",Data Scientist,OpenText,"Makati, Philippines"


### Splitting job details <a class="anchor" id="job"></a>

* The _JobDetails_ column includes the salary, modality, schedule, and level of the job. However, given the limited availability of salary information, it has been omitted from consideration.
* Instances exist where certain variables are absent. To accommodate these cases, adjustments have been made to ensure proper handling of each instance.

In [5]:
combined_data['JobDetails'] = combined_data['JobDetails'].str.removesuffix(" level")
split_details = combined_data['JobDetails'].str.split(" ")

combined_data['Modality'] = ""
combined_data['Schedule'] = ""
combined_data['Level'] = ""

for index, row in combined_data.iterrows():
    if len(split_details[index]) == 3:
        combined_data.at[index, 'Modality'] = split_details[index][0]
        combined_data.at[index, 'Schedule'] = split_details[index][1]
        combined_data.at[index, 'Level'] = split_details[index][2]
    elif len(split_details[index]) == 6:
        combined_data.at[index, 'Modality'] = split_details[index][3]
        combined_data.at[index, 'Schedule'] = split_details[index][4]
        combined_data.at[index, 'Level'] = split_details[index][5]
    elif len(split_details[index]) == 2:
        if split_details[index][0] in ['Remote', 'Hybrid', 'On-site']:
            combined_data.at[index, 'Modality'] = split_details[index][0]
            combined_data.at[index, 'Schedule'] = split_details[index][1]
        else:
            combined_data.at[index, 'Schedule'] = split_details[index][0]
            combined_data.at[index, 'Level'] = split_details[index][1]
    else:
        combined_data.at[index, 'Schedule'] = split_details[index][0]

combined_data = combined_data.drop('JobDetails', axis=1)
combined_data

,Title,JobDescription,JobSkills,Classification,Company,Location,Modality,Schedule,Level
0,Data and Reports Analyst,About the job\nResponsibilities for Reporting ...,"Data Analysis,Critical Thinking\nAdd,Data Anal...",Data Analyst,Dynata,"Davao, Philippines",,Full-time,Mid-Senior
1,Data Analyst,About the job\nRole description:\nMainly provi...,"Data Analysis,Analytical Skills\nAdd,DAX\nAdd,...",Data Analyst,Arcadis,"Makati, Philippines",Hybrid,Full-time,Mid-Senior
2,Data Analyst - People Services,"About the job\nJob Description\n\nAt GHD, we d...","Analytics\nAdd,Change Impact Analysis\nAdd,Dat...",Data Analyst,GHD,"Cebu, Philippines",Hybrid,Full-time,Mid-Senior
3,Insights Analyst,About the job\nIsentia is the Asia-Pacific reg...,"Data Analysis,Google Sheets\nAdd,Google Worksp...",Data Analyst,Isentia,"Pasig, Philippines",Hybrid,Full-time,Associate
4,Risk Data Analyst l Day Shift,"About the job\nWith over 8,000 professionals a...","Data Analysis,Fraud Analysis\nAdd,Risk Analysi...",Data Analyst,MicroSourcing,"National Capital Region, Philippines",Remote,Full-time,Mid-Senior
...,...,...,...,...,...,...,...,...,...
692,Senior IT Officer (Data Scientist),About the job\nImportant Information\n\nThis i...,"ADB\nAdd,Communication\nAdd,Communication Trai...",Data Scientist,Asian Development Bank (ADB),"Manila, Philippines",On-site,Full-time,Mid-Senior
693,Data Scientist,About the job\nRole Responsibilities\n\nWe are...,"Artificial Intelligence (AI)\nAdd,Business Req...",Data Scientist,Standard Chartered Bank,"Makati, Philippines",On-site,Full-time,Entry
694,Data Scientist,About the job\nRoles and Responsibilities\n● D...,"Applications Delivery\nAdd,Cascading Style She...",Data Scientist,Staffing Ninja,Philippines,Remote,Full-time,Mid-Senior
695,Machine Learning Specialist (Python),About the job\nThe opportunity:\n\nWe are look...,"Data Analysis,Analytics\nAdd,Artificial Intell...",Data Scientist,OpenText,"Makati, Philippines",On-site,Full-time,Mid-Senior


### Cleaning list of skills <a class="anchor" id="skills"></a>

* The "JobSkills" column is cleaned by removing the "\nAdd" strings.

In [6]:
combined_data['JobSkills'] = combined_data['JobSkills'].str.replace('\nAdd', '').str.removesuffix(',')
combined_data

,Title,JobDescription,JobSkills,Classification,Company,Location,Modality,Schedule,Level
0,Data and Reports Analyst,About the job\nResponsibilities for Reporting ...,"Data Analysis,Critical Thinking,Data Analytics...",Data Analyst,Dynata,"Davao, Philippines",,Full-time,Mid-Senior
1,Data Analyst,About the job\nRole description:\nMainly provi...,"Data Analysis,Analytical Skills,DAX,Data Analy...",Data Analyst,Arcadis,"Makati, Philippines",Hybrid,Full-time,Mid-Senior
2,Data Analyst - People Services,"About the job\nJob Description\n\nAt GHD, we d...","Analytics,Change Impact Analysis,Data Analytic...",Data Analyst,GHD,"Cebu, Philippines",Hybrid,Full-time,Mid-Senior
3,Insights Analyst,About the job\nIsentia is the Asia-Pacific reg...,"Data Analysis,Google Sheets,Google Workspace,M...",Data Analyst,Isentia,"Pasig, Philippines",Hybrid,Full-time,Associate
4,Risk Data Analyst l Day Shift,"About the job\nWith over 8,000 professionals a...","Data Analysis,Fraud Analysis,Risk Analysis,SQL",Data Analyst,MicroSourcing,"National Capital Region, Philippines",Remote,Full-time,Mid-Senior
...,...,...,...,...,...,...,...,...,...
692,Senior IT Officer (Data Scientist),About the job\nImportant Information\n\nThis i...,"ADB,Communication,Communication Training,Compu...",Data Scientist,Asian Development Bank (ADB),"Manila, Philippines",On-site,Full-time,Mid-Senior
693,Data Scientist,About the job\nRole Responsibilities\n\nWe are...,"Artificial Intelligence (AI),Business Requirem...",Data Scientist,Standard Chartered Bank,"Makati, Philippines",On-site,Full-time,Entry
694,Data Scientist,About the job\nRoles and Responsibilities\n● D...,"Applications Delivery,Cascading Style Sheets (...",Data Scientist,Staffing Ninja,Philippines,Remote,Full-time,Mid-Senior
695,Machine Learning Specialist (Python),About the job\nThe opportunity:\n\nWe are look...,"Data Analysis,Analytics,Artificial Intelligenc...",Data Scientist,OpenText,"Makati, Philippines",On-site,Full-time,Mid-Senior


### Removing duplicates <a class="anchor" id="duplicates"></a>

* The duplicates were removed for each classification.

In [7]:
combined_data = combined_data[['Title', 'Company', 'Location', 'Modality', 'Schedule', 'Level', 'JobSkills', 'JobDescription', 'Classification']]
combined_data = combined_data.drop_duplicates(subset=['Title', 'Company', 'Location', 'Modality', 'Schedule', 'Level', 'JobSkills', 'JobDescription', 'Classification'], keep='last')
combined_data

,Title,Company,Location,Modality,Schedule,Level,JobSkills,JobDescription,Classification
0,Data and Reports Analyst,Dynata,"Davao, Philippines",,Full-time,Mid-Senior,"Data Analysis,Critical Thinking,Data Analytics...",About the job\nResponsibilities for Reporting ...,Data Analyst
1,Data Analyst,Arcadis,"Makati, Philippines",Hybrid,Full-time,Mid-Senior,"Data Analysis,Analytical Skills,DAX,Data Analy...",About the job\nRole description:\nMainly provi...,Data Analyst
3,Insights Analyst,Isentia,"Pasig, Philippines",Hybrid,Full-time,Associate,"Data Analysis,Google Sheets,Google Workspace,M...",About the job\nIsentia is the Asia-Pacific reg...,Data Analyst
4,Risk Data Analyst l Day Shift,MicroSourcing,"National Capital Region, Philippines",Remote,Full-time,Mid-Senior,"Data Analysis,Fraud Analysis,Risk Analysis,SQL","About the job\nWith over 8,000 professionals a...",Data Analyst
5,Junior Data Analyst - Quantitative Data Analys...,ISS | Institutional Shareholder Services,"Makati, Philippines",Hybrid,Full-time,Entry,"ActiveX Data Objects (ADO),Compensation,Comput...",About the job\nLet’s be #BrilliantTogether\n\n...,Data Analyst
...,...,...,...,...,...,...,...,...,...
692,Senior IT Officer (Data Scientist),Asian Development Bank (ADB),"Manila, Philippines",On-site,Full-time,Mid-Senior,"ADB,Communication,Communication Training,Compu...",About the job\nImportant Information\n\nThis i...,Data Scientist
693,Data Scientist,Standard Chartered Bank,"Makati, Philippines",On-site,Full-time,Entry,"Artificial Intelligence (AI),Business Requirem...",About the job\nRole Responsibilities\n\nWe are...,Data Scientist
694,Data Scientist,Staffing Ninja,Philippines,Remote,Full-time,Mid-Senior,"Applications Delivery,Cascading Style Sheets (...",About the job\nRoles and Responsibilities\n● D...,Data Scientist
695,Machine Learning Specialist (Python),OpenText,"Makati, Philippines",On-site,Full-time,Mid-Senior,"Data Analysis,Analytics,Artificial Intelligenc...",About the job\nThe opportunity:\n\nWe are look...,Data Scientist


### Saving job listings file <a class="anchor" id="save"></a>

* The _JobDescription_ was removed as we dont have a use for that in this scenario
* The data frame was saved as an Excel file.
* Since Tableau only supports geolocation at a provincial level, provinces corresponding to specified locations were manually identified. Additionally, location of a few companies with location set as _Philippines_ were determined manually.

In [8]:
combined_data = combined_data.drop('JobDescription', axis=1)
combined_data.to_excel('LinkedIn Recent Job Postings.xlsx', index=False)

### Listing associate skills <a class="anchor" id="list"></a>

* Tableau lacks the capability to count comma-separated values. To address this limitation, the skills were split and listed in a column format.
* The resulting data frame was then saved as a separate Excel file.

In [9]:
def extract_skills(combined_data, classification):
    filtered_data = combined_data[combined_data['Classification'] == classification].copy()
    filtered_data['JobSkills'] = filtered_data['JobSkills'].str.split(",")
    filtered_data = filtered_data.dropna(subset=['JobSkills'])

    df = pd.DataFrame(columns=["Skills", "Job Search"])
    
    for skills_list in filtered_data['JobSkills']:
        df = pd.concat([df, pd.DataFrame({'Skills': skills_list, 'Job Search': classification})], ignore_index=True)

    return df

combined_df_data_analyst = extract_skills(combined_data, 'Data Analyst')
combined_df_data_engineer = extract_skills(combined_data, 'Data Engineer')
combined_df_data_scientist = extract_skills(combined_data, 'Data Scientist')

combined_df = pd.concat([combined_df_data_analyst, combined_df_data_engineer, combined_df_data_scientist], ignore_index=True)
combined_df

,Skills,Job Search
0,Data Analysis,Data Analyst
1,Critical Thinking,Data Analyst
2,Data Analytics,Data Analyst
3,Decision-Making,Data Analyst
4,Object-Oriented Programming (OOP),Data Analyst
...,...,...
6150,Extract,Data Scientist
6151,Transform,Data Scientist
6152,Load (ETL),Data Scientist
6153,Problem Solving,Data Scientist


In [10]:
combined_df.to_excel('Skills.xlsx', index=False)